<a href="https://colab.research.google.com/github/ucfilho/ANN_testing_different/blob/master/ANN_teste_003_out_05_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn.utils import shuffle
import _pickle as cPickle
import gzip

In [0]:



def load_data():
    #Patch='https://github.com/mnielsen/neural-networks-and-deep-learning/data/mnist.pkl.gz'
    #Patch='https://github.com/mnielsen/rmnist/blob/master/data/mnist.pkl.gz'
    #Patch='https://www.kaggle.com/pablotab/mnistpkl.gz'
    #deeplearning.net/data/mnist/mnistpkl.gz
    Patch='mnist.pkl.gz'
    !wget http://deeplearning.net/data/mnist/mnist.pkl.gz
    f = gzip.open(Patch, 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='bytes')
    f.close()
    return training_data, validation_data, test_data

In [0]:
# http://52.32.3.36/post/6
# 

In [0]:
def train(self, X, y, X_val, y_val, nb_epoch, batch_size, eta):
    n = len(X)
    for i in range(nb_epoch):
        epoch_loss = 0
        X, y = shuffle(X, y)
        for j in range(0, n, batch_size):
            X_batch = X[j:j + batch_size]
            y_batch = y[j:j + batch_size]
            loss, grads = self.loss(X_batch, y_batch)
            epoch_loss += loss
            # update parameters
            for param_name in ('W1', 'b1', 'W2', 'b2'):
                self.params[param_name] -= eta * grads[param_name]
        train_acc = self.evaluate(X, y)
        val_acc = self.evaluate(X_val, y_val)
        print("epoch %d / %d: loss %f, train_acc: %f, val_acc: %f" %
              (i + 1, nb_epoch, epoch_loss / n, train_acc, val_acc))

In [0]:
def ReLU(x):
    return np.maximum(0, x)

In [0]:
def dropout(x, dropout_p):
    return x * np.random.binomial([np.ones(x.shape)], 1 - dropout_p)[0] / (1 - dropout_p)

In [0]:
def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

In [0]:
class Model:
    """
    Architecture:
        Flatten -> Dense -> ReLU -> Dropout -> Dense -> SoftMax
    """

    def __init__(self, input_size, hidden_size, output_size, dropout_p):
        self.params = {
            'W1': np.random.randn(input_size, hidden_size) / np.sqrt(input_size),
            'b1': np.zeros((1, hidden_size)),
            'W2': np.random.randn(hidden_size, output_size) / np.sqrt(hidden_size),
            'b2': np.zeros((1, output_size))
        }
        self.dropout_p = dropout_p

    def train(self, X, y, X_val, y_val, nb_epoch, batch_size, eta):
        n = len(X)
        for i in range(nb_epoch):
            epoch_loss = 0
            X, y = shuffle(X, y)
            for j in range(0, n, batch_size):
                X_batch = X[j:j + batch_size]
                y_batch = y[j:j + batch_size]
                loss, grads = self.loss(X_batch, y_batch)
                epoch_loss += loss
                # update parameters
                for param_name in ('W1', 'b1', 'W2', 'b2'):
                    self.params[param_name] -= eta * grads[param_name]
            train_acc = self.evaluate(X, y)
            val_acc = self.evaluate(X_val, y_val)
            print("epoch %d / %d: loss %f, train_acc: %f, val_acc: %f" %
                  (i + 1, nb_epoch, epoch_loss / n, train_acc, val_acc))
    def loss(self, X, y):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']

        n = X.shape[0]

        # feed forward pass
        h1 = ReLU(np.dot(X, W1) + b1)
        h1 = dropout(h1, dropout_p=self.dropout_p)
        out = np.dot(h1, W2) + b2
        probs = softmax(out)

        # loss
        log_probs = -np.log(probs[range(n), y])
        loss = np.sum(log_probs) / n

        # backward pass
        dout = probs
        dout[range(n), y] -= 1
        dh1 = np.dot(dout, W2.T)
        dh1[h1 <= 0] = 0
        dW2 = np.dot(h1.T, dout)
        db2 = np.sum(dout, axis=0, keepdims=True)
        dW1 = np.dot(X.T, dh1)
        db1 = np.sum(dh1, axis=0, keepdims=True)

        grads = {
            'W1': dW1,
            'b1': db1,
            'W2': dW2,
            'b2': db2
        }
        return loss, grads

    def evaluate(self, X, y):
        h1 = ReLU(np.dot(X, self.params['W1']) + self.params['b1'])
        out = np.dot(h1, self.params['W2']) + self.params['b2']
        probs = softmax(out)
        pred = np.argmax(probs, axis=1)
        return sum(pred == y) / X.shape[0]


In [9]:
training_data, validation_data, test_data = load_data()
X, y = training_data
X_val, y_val = validation_data
X_test, y_test = test_data
model = Model(input_size=784, hidden_size=512, output_size=10, dropout_p=0.2)
model.train(X, y, X_val, y_val, nb_epoch=5, eta=0.01, batch_size=10)
print("Test set accuracy", model.evaluate(X_test, y_test))

--2019-10-06 02:04:55--  http://deeplearning.net/data/mnist/mnist.pkl.gz
Resolving deeplearning.net (deeplearning.net)... 132.204.26.28
Connecting to deeplearning.net (deeplearning.net)|132.204.26.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16168813 (15M) [application/x-gzip]
Saving to: ‘mnist.pkl.gz.4’

mnist.pkl.gz.4      100%[===================>]  15.42M  4.39MB/s    in 4.0s    

2019-10-06 02:05:00 (3.86 MB/s) - ‘mnist.pkl.gz.4’ saved [16168813/16168813]

epoch 1 / 5: loss 0.025252, train_acc: 0.967020, val_acc: 0.965500
epoch 2 / 5: loss 0.011288, train_acc: 0.978480, val_acc: 0.974200
epoch 3 / 5: loss 0.007959, train_acc: 0.979860, val_acc: 0.971800
epoch 4 / 5: loss 0.006292, train_acc: 0.988380, val_acc: 0.977800
epoch 5 / 5: loss 0.004992, train_acc: 0.991520, val_acc: 0.977300
Test set accuracy 0.9801
